```{contents}
```
## Authorization

**Authorization** in LangGraph is the mechanism that controls **who is allowed to execute what parts of a graph, with which data, using which tools, and under what conditions**.
It is essential for building **secure, multi-user, enterprise-grade LLM systems**.

---

### **1. Why Authorization Matters in LangGraph**

LangGraph orchestrates:

* LLMs
* External tools
* Private data
* Multi-agent workflows
* Human approvals

Without strong authorization, a user or agent could:

* Call restricted tools
* Access sensitive memory
* Modify protected state
* Execute destructive operations

Authorization enforces **policy-driven control** over every stage of graph execution.

---

### **2. Authorization Layers in LangGraph**

LangGraph does not impose a single built-in auth model.
Instead, it provides **control points** where authorization logic is enforced.

| Layer        | Controlled Assets               |
| ------------ | ------------------------------- |
| API Layer    | Who may invoke a graph          |
| Graph Layer  | Which nodes may execute         |
| Tool Layer   | Which tools may be used         |
| State Layer  | Which fields may be read/write  |
| Memory Layer | Which memories are accessible   |
| Agent Layer  | Which roles an agent may assume |
| Human Layer  | Who can approve/override        |

---

### **3. Authorization Model (Conceptual)**

```
User / Agent
     |
[Authentication]
     |
[Authorization Policy Engine]
     |
LangGraph Execution
     |
Nodes → Tools → State → Memory
```

Authorization is evaluated **before every sensitive action**.

---

### **4. Common Authorization Models**

| Model            | Description                     |
| ---------------- | ------------------------------- |
| RBAC             | Role-Based Access Control       |
| ABAC             | Attribute-Based Access Control  |
| Policy-Based     | Rule engine with conditions     |
| Capability-Based | Token grants specific abilities |
| Contextual       | Based on request context        |

In practice, systems often combine **RBAC + Policy-Based** control.

---

### **5. Authorization in Graph Execution**

### **A. Node-Level Authorization**

```python
def secure_node(state, user):
    if not user.has_permission("execute_sensitive_node"):
        raise PermissionError("Unauthorized")
    return {"result": "ok"}
```

Only authorized users can execute specific nodes.

---

### **B. Tool-Level Authorization**

```python
def restricted_tool(state, user):
    if not user.has_permission("use_payment_api"):
        raise PermissionError()
```

Tools become **privileged operations**.

---

### **C. State-Level Authorization**

```python
def update_salary(state, user):
    if not user.has_permission("write_salary"):
        raise PermissionError()
```

Controls access to **sensitive state fields**.

---

### **D. Memory-Level Authorization**

```python
if not user.has_permission("read_private_memory"):
    block_memory_access()
```

Protects long-term memory and vector stores.

---

### **6. Human-in-the-Loop Authorization Gates**

LangGraph supports **approval nodes** for privileged operations.

```
Analyze → Propose Action → Human Approval → Execute
```

```python
if state["requires_approval"]:
    interrupt_for_human()
```

This enforces **two-layer security**: policy + human.

---

### **7. Production Authorization Architecture**

```
Client
  |
Auth Server (OAuth / SSO)
  |
Policy Engine (OPA / Custom)
  |
LangGraph Runtime
  |
State Store + Tools + Memory
```

All requests carry an **authorization context**.

---

### **8. Advanced Controls**

| Feature           | Purpose                  |
| ----------------- | ------------------------ |
| Per-node policies | Fine-grained control     |
| Execution scopes  | Limit graph capabilities |
| Sandboxing        | Restrict tool execution  |
| Audit logs        | Trace who did what       |
| Immutable history | Compliance               |
| Kill-switch       | Emergency shutdown       |

---

### **9. Example: Role-Based Graph Control**

```python
def router(state, user):
    if user.role == "admin":
        return "admin_node"
    elif user.role == "user":
        return "user_node"
    else:
        raise PermissionError()
```

---

### **10. Security Guarantees**

| Guarantee        | Result               |
| ---------------- | -------------------- |
| Least privilege  | Minimal access       |
| Defense in depth | Multiple checkpoints |
| Traceability     | Full audit           |
| Compliance       | Regulatory readiness |

---

### **11. Mental Model**

Authorization in LangGraph is a **policy-driven execution firewall** that surrounds:

> **Graphs · Nodes · Tools · State · Memory · Agents · Humans**

and ensures **only approved actions occur** at runtime.


### Demonstration

In [4]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# -------------------- User & State --------------------

class User:
    def __init__(self, role):
        self.role = role
    
    def has_permission(self, perm):
        perms = {
            "admin": {"view", "modify", "pay"},
            "user": {"view"}
        }
        return perm in perms[self.role]

class State(TypedDict):
    data: str
    result: str
    user: object

# -------------------- Secure Nodes --------------------

def entry_node(state):
    return {}   # entry must return dict

def view_data(state):
    if not state["user"].has_permission("view"):
        raise PermissionError()
    return {"result": f"Viewing: {state['data']}"}

def modify_data(state):
    if not state["user"].has_permission("modify"):
        raise PermissionError()
    return {"result": f"Modified: {state['data']}"}

def payment_tool(state):
    if not state["user"].has_permission("pay"):
        raise PermissionError()
    return {"result": "Payment executed"}

# -------------------- Router Function (NOT a node) --------------------

def route(state):
    return "modify" if state["user"].role == "admin" else "view"

# -------------------- Build Graph --------------------

builder = StateGraph(State)

builder.add_node("entry", entry_node)
builder.add_node("view", view_data)
builder.add_node("modify", modify_data)
builder.add_node("pay", payment_tool)

builder.set_entry_point("entry")

builder.add_conditional_edges("entry", route, {
    "modify": "modify",
    "view": "view"
})

builder.add_edge("modify", "pay")
builder.add_edge("view", END)
builder.add_edge("pay", END)

graph = builder.compile()

# -------------------- Execute --------------------

admin = User("admin")
user = User("user")

print("Admin run:")
print(graph.invoke({"data": "Report", "user": admin, "result": ""}))

print("\nUser run:")
print(graph.invoke({"data": "Report", "user": user, "result": ""}))


Admin run:
{'data': 'Report', 'result': 'Payment executed', 'user': <__main__.User object at 0x000002028788CD40>}

User run:
{'data': 'Report', 'result': 'Viewing: Report', 'user': <__main__.User object at 0x000002028788F050>}
